In [ ]:
#!/usr/bin/env python

"""
Takes output from DESeq2 and makes pretty plots / analysis of the output.
For reference I can use:
http://pandas.pydata.org/pandas-docs/stable/visualization.html
http://www.labri.fr/perso/nrougier/teaching/matplotlib/#bar-plots
"""

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context="paper", font="monospace")

#####may not do this section but just in case here is the beginning of it. otherwise proceed to male/female...
####bar plot of gene families
####read in categories of functions
functions_table = pd.read_csv("pathway_analysis/categories_panther.csv")
functions_table.shape
####read in larger panther analysis file to merge unassigned genes with categorized genes
panther_table = pd.read_csv("pathway_analysis/panther/Panther_degenes_3G1G.csv")
panther_table.shape
functions= pd.merge(functions_table, panther_table, how="outer", on=["gene"])
functions.shape
####functions.to_csv("pathway_analysis/panther/all_functions.csv")
category_count = pd.Series(functions["category"])
categories = category_count.value_counts()
print(categories)
print(sum(categories))

#Male to Female bias check:

In [ ]:
#read in DESeq2 output for my samples and add fbgn names
deseq_df = pd.read_csv("DESeq2/deseq2_sig.05_3G1G.csv")
header_row=['gene','fbgene']
fbgn = pd.read_table("sample_stats/symbols_and_transcripts.tsv", names=header_row)
diff_df = pd.merge(deseq_df, fbgn, how='left', on='gene')
#read in male/female miccroarray data from PMID: 19216785 paper
mf_comp = pd.read_excel("sample_stats/male_fem_comparison/sex_time_difff.xls", sheetname=3, header=3)
mf_comp = mf_comp.rename(columns = {'FBGN' : 'fbgene'})

In [ ]:
mf_comp.columns

In [ ]:
#check that no genes are missing fbgene names
mask=False
for col in diff_df.columns: mask = mask | diff_df['fbgene'].isnull()
dfnulls = diff_df[mask]
print(dfnulls)

In [ ]:
mf_genes = pd.merge(diff_df, mf_comp, how='inner', on='fbgene')
mf_genes.drop(mf_genes.columns[[1,3,4,5,6,9,11,12]], axis=1, inplace=True)
mf_genes[['Fold Change (M/F)']] = mf_genes[['Fold Change (M/F)']].astype(float)
#mf_genes.columns

In [ ]:
print(mf_genes.shape)
mf_genes.sort(columns='Sex-bias', ascending=False, inplace=True)
mf_gene = mf_genes.set_index('gene')
mf_gene.index.name = None
mf_gene

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
#bar plot of m/f expression
#plot the data
color_list = ['orange','g']
ax = mf_gene[['log2FoldChange','Fold Change (M/F)']].plot(kind='bar', color=color_list, title ="Sex-specific gene expression",figsize=(15,10), fontsize=16)
ax.set_ylabel("fold change",fontsize=12)
#plot 0 line black
plt.axhline(0, color='k')
#make the legend
orange = mpatches.Patch(color='orange', label='3G samples')
green = mpatches.Patch(color='green', label='sex specific')
red = mpatches.Patch(color='pink', label= 'female genes')
blue = mpatches.Patch(color='blue', label= 'male genes', alpha = 0.2)
plt.legend(loc='upper left', shadow=True, fontsize='x-large', handles=[orange,green,blue,red], frameon=True)
#add shading to the background
a = -1
b = 4.5
plt.axvspan(a, b, color = 'blue', alpha=0.2)
c = 4.5
d = 20
plt.axvspan(c, d, color = 'pink', alpha=0.4)
#save and plot figure
plt.savefig('highres.pdf')
plt.show()

In [ ]:
mf_genes.groupby('Sex-bias')[['log2FoldChange','Fold Change (M/F)']].corr(method='pearson')

In [ ]:
#look at sex-specific genes for male or female at sample level
sample_counts = pd.read_table("DESeq2/merged_counts_eXpress_TPM100_genelevel.tsv")
samples = pd.merge(sample_counts, fbgn, how='left', on='gene')
samples.head()

In [ ]:
mf_bysamples = pd.merge(samples, mf_comp, how='inner', on='fbgene')

In [ ]:
mf_bysamples.shape

In [ ]:
mf_bysamples.sort(columns='Sex-bias', ascending=False, inplace=True)
mf_bysamples.head()

In [ ]:
male_specific = mf_bysamples[(mf_bysamples['Sex-bias'] == 'Male-biased')]
female_specific = mf_bysamples[(mf_bysamples['Sex-bias'] == 'Female-biased')]

In [ ]:
#I'm exporting this to excel for laziness in barplot production
print(male_specific.sum())
print(female_specific.sum())

In [ ]:
male_specific.sum()

#Stress comparison:

I am comparing the 3G DE to the various stressor induced DE from the paper, "Diversity and dynamics of the Drosophila transcriptome" and showing that 3G ellicits a unique response. 

I would like to generate a heat map comparing the values.

#####Here are my notes from the preliminary comparison:
Do the effects resemble common stress effects? Do they resemble any of the other stressors more than other? Which genes / pathways are specific to 3G?

“we find a homogeneous response to environmental stressors"

Direction of change is consistent across all treatments



Upregulated:
Response to Stimulus, GO:0050896
Lyzosymes ( >10fold)
cytochrome P450s
mitochontrial components mt:ATPase6, mt:CoI, mt:CoIII

Downregulated:
egg-shell, yolk, seminal fluid except in heatshock and cold2 treatments because collection was too early for these late response genes (so mine will also be downregulated)

At home: open and look for these above groups in the file: “Panther DE genes 3G1G in the pathway analysis folder. make a new spreadsheet in the stress comparison folder quantifying the relationships between our genes and the stress genes from the Brown study. Think up the best way to visualize the results so that tomorrow I can make a table and write up the findings to email to Ravi.


NEXT: color code based upon pos or neg

RESULTS:
782 significant genes in our study were also reported in the Brown stress study

Resp. to Stim. genes:
7 found in .05 group: LysD, Fst, Tequila, DopEcR, Hsp67Bb, Rdl, Cyp6a8
4 upregulated, but Rdl and DopEcr are both -0.8, Cyp6a2 is -1. Not too far from 0

Downregulated genes: not in sig list

Lysosomal genes were upregulated but don’t appear to be so in the other paper.
Cy P450 genes are both up and down regulated.

In [ ]:
#read in excel spreadsheet of 3G genes also found in stress study
stress_genes = pd.read_excel("stress comparison/matching_genes.xlsx")
stress_genes.drop(stress_genes.columns[[2]], axis=1, inplace=True)
#read in functions for 3G genes to merge with stress data file
functions = pd.read_csv('pathway_analysis/panther/Panther_degenes_3G1G.csv')
functions.drop(functions.columns[[1,2,3,4,5,6,7,8,9,10]], axis=1, inplace=True)

In [ ]:
#merge these files together and clean them up
stress_df = pd.merge(stress_genes, functions, how='left', on='gene')
stress_df = stress_df.set_index('gene')
stress_df.index.name = None
stress_df.drop(stress_df.columns[[2,4,6,8,10,12,14,16,18,20,22]], axis=1, inplace=True)

In [ ]:
fold_change = stress_df.ix[:,:'Zn_4.5mM Fold Change']
fold_change.head()

In [ ]:
#stress_df.to_csv('stress_comparison.csv')
#fold_change.describe()
dfFC = fold_change.replace(1, np.nan)
dfFC.head()

In [ ]:
correlation = dfFC.corr(method='spearman')
#correlation.to_csv('stress comparison/stress_spearman.csv')

In [ ]:
sns.heatmap(fold_change, vmax=10, vmin=-10, yticklabels=False)

In [ ]:
sns.heatmap(dfFC, vmax=10, vmin=-10, yticklabels=False)

In [ ]:
spear = fold_change.corr(method='spearman')

clustmap = sns.clustermap(spear)
clustmap.savefig('clustermap.png')

In [ ]:
spear = dfFC.corr(method='spearman')

clustmap = sns.clustermap(spear)
clustmap.savefig('clustermap.pdf')  #this one is used in the paper

In [ ]:
#let's see if I can duplicate the heatmap in the celnicker paper
celnicker = pd.read_excel('stress comparison/Supplementary table 9.xlsx')
celnicker = celnicker.set_index('Gene_Name')
celnicker.index.name = None
celnicker.drop(celnicker.columns[[1,3,5,7,9,11,13,15,17,19,21]], axis=1, inplace=True)

In [ ]:
celnicker.head()

In [ ]:
sns.heatmap(celnicker, vmax=10, vmin=-10)

###Counting genes in categories
I am counting up the genes that have been categorized to check that each gene has been used once and that no genes are missing.

In [ ]:
#read in csv sheets for the various gene categories
defense = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=6, header=0)
defense['category'] = 'defense'
binding = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=16, header=0)
binding['category'] = 'binding'
cuticle = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=8, header=0)
cuticle['category'] = 'cuticle'
metabolic = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=17, header=0)
metabolic['category'] = 'metabolic'
other = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=18, header=0)
other['category'] = 'other'
oxidoreductase = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=7, header=0)
oxidoreductase['category'] = 'redox'
proteolysis = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=13, header=0)
proteolysis['category'] = 'proteolysis'
ribosome = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=14, header=0)
ribosome['category'] = 'translation'
transferase = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=15, header=0)
transferase['category'] = 'transferase'
transport = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=9, header=0)
transport['category'] = 'transport'
sodium = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=10, header=0)
sodium['category'] = 'sodium'
potassium = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=11, header=0)
potassium['category'] = 'potassium'
calcium = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=12, header=0)
calcium['category'] = 'calcium'
uncharacterized = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=19, header=0)
uncharacterized['category'] = 'uncharacterized'

In [ ]:
#combine sheets into one table of categories
frames = [defense, binding, cuticle, metabolic, other, oxidoreductase, proteolysis, ribosome, transferase, transport, sodium, potassium, calcium, uncharacterized]
categories = pd.concat(frames)

In [ ]:
categories.shape

In [ ]:
#count how many genes are in each group
group_count = pd.Series(categories["category"])
counts = (group_count.value_counts())
print(counts)

In [ ]:
#check to see if any genes are duplicated
genes = categories["GENE"]
categories[genes.isin(genes[genes.duplicated()])].sort("GENE")
#all checks out, so the list is complete!

In [ ]:
#combine the categories with the total results output by DESeq2
#for a single csv of results
de_df = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=1, header=0)

In [ ]:
categories = categories.rename(columns = {'GENE' : 'Gene', 'MOLECULAR FUNCTION' : 'GO_molecular_function', 'BIOLOGICAL PROCESS' : 'GO_biological_process'})
#combined_df = pd.merge(de_df, categories, on='Gene', how='left')
#combined_df.drop(combined_df.columns[[8,9]], axis=1, inplace=True)

In [ ]:
#combined_df.head()
combined_df.shape
combined_df.describe()

In [ ]:
#combined_df.to_csv('All_DE_genes.tsv', sep='\t')

In [ ]:
#use this code as a work around for errors like:
#UnicodeEncodeError: 'ascii' codec can't encode character u'\xa0' in position

#import sys
#reload(sys)
#sys.setdefaultencoding("utf8")

In [ ]:
#output other files into tsv format
primers = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=21, header=0)
primers = primers[np.isfinite(primers['SL_no'])]
stress_comparison = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=22, header=0)
eclosion_rates = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=23, header=1)
read_details = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=24, header=0)
software_details = pd.read_excel("manuscript/Supplemental_table1.xlsx", sheetname=25, header=0)

In [ ]:
primers.to_csv('primers.txt', sep='\t')
stress_comparison.to_csv('stress_comparison.txt', sep='\t')
eclosion_rates.to_csv('eclosion_rates.txt', sep='\t')
read_details.to_csv('read_details.txt', sep='\t')
software_details.to_csv('software_details.txt', sep='\t')


In [ ]:
jh = categories[categories['GO_molecular_function'].str.contains('serine')] | categories['category'] == 'proteolysis'
jh

In [ ]:
categories.columns